# AWS Re:Invent  Autogluon Workshop
### This workshop will demonstrate a machine learning problem solved by autogluon.
* Use the documentation of autogluon and the different tutorials [here](https://auto.gluon.ai/stable/index.html).

In [1]:
# Installation
!pip3 install -U pip
!pip3 install -U setuptools wheel

# Install the proper version of PyTorch following https://pytorch.org/get-started/locally/
!pip3 install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchtext==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu113

!pip3 install "mxnet_cu101<2.0.0, >=1.7.0"    
    
!pip3 install autogluon

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3
    Uninstalling pip-22.3:
      Successfully uninstalled pip-22.3
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-

In [2]:
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [4]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [5]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20221122_095451/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221122_095451/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify po

In [6]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [7]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523
}


In [8]:
predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523
}


{'accuracy': 0.8374449790152523}

In [9]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.157144,0.061006,0.583327,0.157144,0.061006,0.583327,1,True,5
1,CatBoost,0.842461,0.85,0.037355,0.009213,1.220509,0.037355,0.009213,1.220509,1,True,7
2,RandomForestEntr,0.841130,0.83,0.156240,0.059861,0.470254,0.156240,0.059861,0.470254,1,True,6
3,LightGBM,0.839799,0.85,0.029294,0.005735,0.252078,0.029294,0.005735,0.252078,1,True,4
4,XGBoost,0.837445,0.87,0.048340,0.007246,0.224522,0.048340,0.007246,0.224522,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.051539,0.007804,0.620225,0.003199,0.000558,0.395703,2,True,14
6,LightGBMXT,0.836421,0.83,0.017661,0.005626,1.071510,0.017661,0.005626,1.071510,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.166816,0.057844,0.468804,0.166816,0.057844,0.468804,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.188453,0.058337,0.463114,0.188453,0.058337,0.463114,1,True,9
9,LightGBMLarge,0.828949,0.83,0.033231,0.006524,0.355061,0.033231,0.006524,0.355061,1,True,13


### Using hyperparameter settings
#### You can find [here](https://auto.gluon.ai/dev/api/autogluon.tabular.models.html) some models and which hyperparameters you can set.

In [10]:
import autogluon.core as ag

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


In [11]:
nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

rf_options = {
    'n_estimators': 400,
    'max_leaf_nodes' : ag.space.Int(lower=500, upper=15000, default=2000),
}

cat_options = {
    'learning_rate' : ag.space.Real(0.0,0.2,default=0.05),
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                   'RF': rf_options,
                   'CAT': cat_options,
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~10 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='acc').fit(
    train_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221122_095507/"
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221122_095507/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	0.8756	 = Validation score   (accuracy)
	0.48s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.8764	 = Validation score   (accuracy)
	0.51s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.8756	 = Validation score   (accuracy)
	0.62s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.8186	 = Validation score   (accuracy)
	0.51s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.8742	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: RandomForest ... Tuning model for up to 134.94s of the 596.64s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: RandomForest/T1 ...
	0.8637	 = Validation score   (accuracy)
	2.94s	 = Training   runtime
	1.11s	 = Validation runtime
Fitted model: RandomForest/T2 ...
	0.8608	 = Validation score   (accuracy)
	2.09s	 = Training   runtime
	0.78s	 = Validation runtime
Fitted model: RandomForest/T3 ...
	0.8546	 = Validation score   (accuracy)
	1.7s	 = Training   runtime
	0.62s	 = Validation runtime
Fitted model: RandomForest/T4 ...
	0.8547	 = Validation score   (accuracy)
	1.67s	 = Training   runtime
	0.61s	 = Validation runtime
Fitted model: RandomForest/T5 ...
	0.8587	 = Validation score   (accuracy)
	1.98s	 = Training   runtime
	0.75s	 = Validation runtime
Hyperparameter tuning model: CatBoost ... Tuning model for up to 134.94s of the 577.95s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: CatBoost/T1 ...
	0.8766	 = Validation score   (accuracy)
	12.52s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: CatBoost/T2 ...
	0.877	 = Validation score   (accuracy)
	5.99s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: CatBoost/T3 ...
	0.8776	 = Validation score   (accuracy)
	11.22s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: CatBoost/T4 ...
	0.8778	 = Validation score   (accuracy)
	10.49s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: CatBoost/T5 ...
	0.877	 = Validation score   (accuracy)
	13.56s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ... Tuning model for up to 134.94s of the 523.95s of remaining time.
Fitted model: NeuralNetTorch/ecf5e_00000 ...
	0.853	 = Validation score   (accuracy)
	32.86s	 = Training   runtime
	0.28s	 = Validation runtime
Fitted model: NeuralNetTorch/ecf5e_00001 ...
	0.8568	 = Validation score   (accuracy)
	31.29s	 =

In [12]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8748080663322756
Evaluations on test data:
{
    "accuracy": 0.8748080663322756
}


In [13]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost/T5,0.875934,0.877000,0.021490,0.015692,13.562973,0.021490,0.015692,13.562973,1,True,15
1,LightGBM/T2,0.875934,0.876400,0.032944,0.016001,0.514403,0.032944,0.016001,0.514403,1,True,2
2,CatBoost/T4,0.875115,0.877800,0.019815,0.013454,10.492069,0.019815,0.013454,10.492069,1,True,14
3,CatBoost/T3,0.875115,0.877600,0.020960,0.013583,11.221760,0.020960,0.013583,11.221760,1,True,13
4,WeightedEnsemble_L2,0.874808,0.878000,0.653411,0.297256,46.693883,0.006920,0.007200,3.341228,2,True,21
5,CatBoost/T1,0.874706,0.876600,0.021744,0.016599,12.521850,0.021744,0.016599,12.521850,1,True,11
6,LightGBM/T1,0.874296,0.875600,0.036829,0.017051,0.482210,0.036829,0.017051,0.482210,1,True,1
7,CatBoost/T2,0.873989,0.877000,0.017154,0.011777,5.992388,0.017154,0.011777,5.992388,1,True,12
8,LightGBM/T3,0.873887,0.875600,0.046078,0.021614,0.622917,0.046078,0.021614,0.622917,1,True,3
9,LightGBM/T5,0.872249,0.874200,0.038709,0.018659,0.576578,0.038709,0.018659,0.576578,1,True,5


### Using ensembling techniques 

In [14]:
nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

rf_options = {
    'n_estimators': 400,
    'max_leaf_nodes' : ag.space.Int(lower=500, upper=15000, default=2000),
}

cat_options = {
    'learning_rate' : ag.space.Real(0.0,0.2,default=0.05),
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                   'RF': rf_options,
                   'CAT': cat_options,
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~10 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='acc').fit(
    train_data, time_limit=time_limit, #auto_stack=True,
    num_bag_folds=5, num_bag_sets=2, num_stack_levels=2,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221122_095737/"
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221122_095737/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L1/T1 ...
	0.8713	 = Validation score   (accuracy)
	0.53s	 = Training   runtime
	0.06s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	0.8715	 = Validation score   (accuracy)
	0.54s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	0.8694	 = Validation score   (accuracy)
	0.63s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	0.8193	 = Validation score   (accuracy)
	0.52s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	0.8681	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.03s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L1 ... Tuning model for up to 11.99s of the 596.14s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: RandomForest_BAG_L1/T1 ...
	0.8638	 = Validation score   (accuracy)
	4.49s	 = Training   runtime
	1.65s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ... Tuning model for up to 11.99s of the 587.77s of remaining time.


...

	Stopping HPO to satisfy time limit...
Fitted model: LightGBM_BAG_L3/T1 ...
	0.8811	 = Validation score   (accuracy)
	0.89s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L3/T2 ...
	0.8825	 = Validation score   (accuracy)
	0.94s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: LightGBM_BAG_L3/T3 ...
	0.8827	 = Validation score   (accuracy)
	1.23s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L3 ... Tuning model for up to 4.73s of the 101.47s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: RandomForest_BAG_L3/T1 ...
	0.8725	 = Validation score   (accuracy)
	18.85s	 = Training   runtime
	2.15s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L3 ... Tuning model for up to 4.73s of the 77.35s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 61.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L3/T1 ...
	0.8821	 = Validation score   (accuracy)
	3.69s	 = Training   runtime
	0.03s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L3 ... Tuning model for up to 4.73s of the 73.42s of remaining time.
NaN or Inf found in input tensor.
2022-11-22 10:06:30,317	INFO stopper.py:363 -- Reached timeout of 3.7835224056243906 seconds. Stopping all trials.
Fitting model: LightGBM_BAG_L3/T1 ... Training model for up to 66.66s of the 66.64s of remaining time.
	Fitting 4 child models (S1F2 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.8744	 = Validation score   (accuracy)
	5.22s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: LightGBM_BAG_L3/T2 ... Training model for up to 58.69s of the 58.67s of remaining time.
	Fitting 4 child models (S1F2 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.8752	 = Validation score   (a

In [15]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8745009724639164
Evaluations on test data:
{
    "accuracy": 0.8745009724639164
}


In [16]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L2/T5,0.876446,0.873672,6.548983,6.827475,203.032670,0.245272,0.208085,3.938540,2,True,16
1,CatBoost_BAG_L2/T1,0.875729,0.875234,6.394531,6.728380,228.955591,0.090820,0.108991,29.861461,2,True,18
2,WeightedEnsemble_L3,0.875729,0.875234,6.396495,6.791907,245.474826,0.001964,0.063527,16.519235,3,True,23
3,LightGBM_BAG_L2/T2,0.875729,0.873698,6.482572,6.728957,202.482234,0.178861,0.109567,3.388104,2,True,13
4,LightGBM_BAG_L2/T3,0.875729,0.873391,6.553095,6.800457,202.925216,0.249384,0.181068,3.831086,2,True,14
5,LightGBM_BAG_L2/T1,0.875422,0.874235,6.515791,6.830537,202.602746,0.212080,0.211147,3.508616,2,True,12
6,NeuralNetTorch_BAG_L2/T1,0.875422,0.872981,7.911817,8.051803,234.419336,1.608106,1.432413,35.325206,2,True,19
7,LightGBM_BAG_L1/T1,0.874910,0.873288,0.347302,0.438626,2.869899,0.347302,0.438626,2.869899,1,True,1
8,LightGBM_BAG_L3/T1,0.874910,0.874389,13.352954,14.564839,419.013834,0.185335,0.276117,5.217258,3,True,24
9,CatBoost_BAG_L1/T1,0.874808,0.874517,0.178349,0.187852,89.889010,0.178349,0.187852,89.889010,1,True,7


#### Extra automation

In [17]:
nn_options = {}
gbm_options = {}
rf_options = {}
cat_options = {}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                   'RF': rf_options,
                   'CAT': cat_options,
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~10 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='acc').fit(
    train_data, time_limit=time_limit, auto_stack=True, presets='best_quality',
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    sample_weight='balanced_weight',
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221122_100812/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221122_100812/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-

  0%|          | 0/5 [00:00<?, ?it/s]

[1000]	valid_set's binary_error: 0.125281


	Stopping HPO to satisfy time limit...
Fitted model: LightGBM_BAG_L1/T1 ...
	0.8747	 = Validation score   (accuracy)
	1.14s	 = Training   runtime
	0.06s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	0.8745	 = Validation score   (accuracy)
	1.39s	 = Training   runtime
	0.06s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	0.8755	 = Validation score   (accuracy)
	1.28s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	0.8747	 = Validation score   (accuracy)
	6.36s	 = Training   runtime
	0.39s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L1 ... Tuning model for up to 16.87s of the 587.03s of remaining time.
	No hyperparameter search space specified for RandomForest. Skipping HPO. Will train one model based on the provided hyperparameters.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/autogluon/core/trainer/abstract_trainer.py", line 138

  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 284.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L1/T1 ...
	0.8723	 = Validation score   (accuracy)
	13.42s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 16.87s of the 565.4s of remaining time.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
2022-11-22 10:09:02,479	INFO stopper.py:363 -- Reached timeout of 13.493324695229532 seconds. Stopping all trials.
Fitted model: NeuralNetTorch_BAG_L1/T1 ...
	0.8475	 = Validation score   (accuracy)
	7.86s	 = Training   runtime
	0.15s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T2 ...
	0.8426	 = Validation score   (accuracy)
	8.4s	 = Training   runtime
	0.15s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T3 ...
	0.8356	 = Validation score   (accuracy)
	8.35s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM_BAG_L1/T1 ... Training 

In [18]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8753198894462074
Evaluations on test data:
{
    "accuracy": 0.8753198894462074
}


In [19]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1/T1,0.876139,0.873391,0.250188,0.354206,178.363245,0.250188,0.354206,178.363245,1,True,5
1,LightGBM_BAG_L1/T1,0.876036,0.874543,0.810949,0.935043,11.969077,0.810949,0.935043,11.969077,1,True,1
2,LightGBM_BAG_L1/T4,0.875525,0.873800,4.690636,5.198068,35.623856,4.690636,5.198068,35.623856,1,True,4
3,LightGBM_BAG_L1/T3,0.875422,0.873979,0.887022,0.781623,11.866903,0.887022,0.781623,11.866903,1,True,3
4,LightGBM_BAG_L1/T2,0.875320,0.874850,0.796434,0.744634,12.435519,0.796434,0.744634,12.435519,1,True,2
5,WeightedEnsemble_L2,0.875320,0.874875,2.782690,2.298138,239.135580,0.002541,0.074118,8.975713,2,True,9
6,NeuralNetTorch_BAG_L1/T1,0.863138,0.857933,1.983714,1.479386,217.724348,1.983714,1.479386,217.724348,1,True,6
7,NeuralNetTorch_BAG_L1/T2,0.858327,0.854017,1.684031,1.268226,87.562599,1.684031,1.268226,87.562599,1,True,7
8,NeuralNetTorch_BAG_L1/T3,0.843689,0.835619,0.211390,0.093792,8.351446,0.211390,0.093792,8.351446,1,True,8
